In [41]:
# CODE BLOCK 1: IMPORTS
import numpy as np
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
# import en_core_web_lg
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
pd.set_option('display.max_colwidth', None)
import wordsegment
from wordsegment import load, segment
from google_spelling_checker_dict import * # file downloaded from Chenglong public repo shared on Kaggle forum

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spiro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
def load_data(train_path, test_path, attributes_path, product_descriptions_path, train_len = None, bool_dev = False):
    df_train = pd.read_csv(train_path, encoding = 'ISO-8859-1')
    df_test = pd.read_csv(test_path, encoding = 'latin-1')
    df_attr = pd.read_csv(attributes_path, encoding = 'ISO-8859-1')
    df_pro_desc = pd.read_csv(product_descriptions_path, encoding = 'latin-1')

    if train_len == None:
        len_training = df_train.shape[0]
    else:
        len_training = num_samples
    df_train = df_train[:len_training]
    df_test = df_test[len_training:]
    pro_desc = df_pro_desc[:len_training]

    df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
    df_all = pd.merge(df_all, pro_desc, how='left', on='product_uid')
    
    if bool_dev:
        return df_all[:400], df_attr[:400]
    else:
        return df_all, df_attr

train_path = 'home-depot-product-search-relevance/train.csv/train.csv'
test_path = 'home-depot-product-search-relevance/test.csv/test.csv'
attributes_path = 'home-depot-product-search-relevance/attributes.csv/attributes.csv'
product_descriptions_path = 'home-depot-product-search-relevance/product_descriptions.csv/product_descriptions.csv'

# df_all, df_attr = load_data(train_path, test_path, attributes_path, product_descriptions_path, bool_dev = True)
df_all, df_attr = load_data(train_path, test_path, attributes_path, product_descriptions_path, bool_dev = True)

In [46]:
def spell_check(df_all, dictionary):
    
    columns = ['product_title', 'product_description']
    for column in columns:    
        for phrase in df_all[column]:
            for key in dictionary.keys():
                phrase.replace(key, dictionary[key])
    
    return df_all

In [47]:
spell_check(df_all, spelling_checker_dict)

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws"
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws"
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 Tugboat Wood and Concrete Coating,deck over,3.00,"BEHR Premium Textured DECKOVER is an innovative solid color coating. It will bring your old, weathered wood or concrete back to life. The advanced 100% acrylic resin formula creates a durable coating for your tired and worn out deck, rejuvenating to a whole new look. For the best results, be sure to properly prepare the surface using other applicable BEHR products displayed above.California residents: see&nbsp;Proposition 65 informationRevives wood and composite decks, railings, porches and boat docks, also great for concrete pool decks, patios and sidewalks100% acrylic solid color coatingResists cracking and peeling and conceals splinters and cracks up to 1/4 in.Provides a durable, mildew resistant finishCovers up to 75 sq. ft. in 2 coats per gallonCreates a textured, slip-resistant finishFor best results, prepare with the appropriate BEHR product for your wood or concrete surfaceActual paint colors may vary from on-screen and printer representationsColors available to be tinted in most storesOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT"
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Kit in Chrome (Valve Not Included),rain shower head,2.33,"Update your bathroom with the Delta Vero Single-Handle Shower Faucet Trim Kit in Chrome. It has a sleek, modern and minimalistic aesthetic. The MultiChoice universal valve keeps the water temperature within +/-3 degrees Fahrenheit to help prevent scalding.California residents: see&nbsp;Proposition 65 informationIncludes the trim kit only, the rough-in kit (R10000-UNBX) is sold separatelyIncludes the handleMaintains a balanced pressure of hot and cold water even when a valve is turned on or off elsewhere in the systemDue to WaterSense regulations in the state of New York, please confirm your shipping zip code is not restricted from use of items that do not meet WaterSense qualifications"
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Kit in Chrome (Valve Not

In [48]:
def remove_words_in_brackets(df_all):
    '''
    This is only used on the title!
    '''
    
    df_all["product_title"] = df_all["product_title"].str.replace(r"(\(.*?\))", "",regex=True)
    
    return df_all

In [49]:
df_all = remove_words_in_brackets(df_all)

In [50]:
def space_before_capitals(df_all):
    load()
    columns = ['product_description']
    for column in columns:
        df_all[column] = df_all[column].str.replace(r"([a-z])([A-Z])",r"\1 \2", regex = True)
    
    return df_all

In [51]:
df_all = space_before_capitals(df_all)

In [52]:
df_all.head()

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projects Stronger than angled nailing or screw fastening alone Help ensure joints are consistently straight and strong Dimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steel Galvanized for extra corrosion resistance Install with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws"
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projects Stronger than angled nailing or screw fastening alone Help ensure joints are consistently straight and strong Dimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steel Galvanized for extra corrosion resistance Install with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws"
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 Tugboat Wood and Concrete Coating,deck over,3.00,"BEHR Premium Textured DECKOVER is an innovative solid color coating. It will bring your old, weathered wood or concrete back to life. The advanced 100% acrylic resin formula creates a durable coating for your tired and worn out deck, rejuvenating to a whole new look. For the best results, be sure to properly prepare the surface using other applicable BEHR products displayed above.California residents: see&nbsp;Proposition 65 information Revives wood and composite decks, railings, porches and boat docks, also great for concrete pool decks, patios and sidewalks100% acrylic solid color coating Resists cracking and peeling and conceals splinters and cracks up to 1/4 in.Provides a durable, mildew resistant finish Covers up to 75 sq. ft. in 2 coats per gallon Creates a textured, slip-resistant finish For best results, prepare with the appropriate BEHR product for your wood or concrete surface Actual paint colors may vary from on-screen and printer representations Colors available to be tinted in most stores Online Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT"
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Kit in Chrome,rain shower head,2.33,"Update your bathroom with the Delta Vero Single-Handle Shower Faucet Trim Kit in Chrome. It has a sleek, modern and minimalistic aesthetic. The Multi Choice universal valve keeps the water temperature within +/-3 degrees Fahrenheit to help prevent scalding.California residents: see&nbsp;Proposition 65 information Includes the trim kit only, the rough-in kit (R10000-UNBX) is sold separately Includes the handle Maintains a balanced pressure of hot and cold water even when a valve is turned on or off elsewhere in the system Due to Water Sense regulations in the state of New York, please confirm your shipping zip code is not restricted from use of items that do not meet Water Sense qualifications"
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Kit in Chrome,shower

In [53]:
def remove_stopwords(df_all, df_attr):
    '''
    This removes stopwords from title, product description, attributes, and search term.
    
    df_all: pandas dataframe incl product_title, search_term, and product_description as columns
    df_attr: pandas dataframe incl 
    '''
    
    stop_words_incl_in = set(stopwords.words(("english")))
    stop_words = set([word for word in stop_words_incl_in if word != 'in']) # removing in from stop_words since dimensions use in. to denote inches
    
    columns = ['search_term','product_title', 'product_description']
    for column in columns:
        df_all[column] = df_all[column].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    return df_all

In [54]:
remove_stopwords(df_all, df_attr)

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not angles make joints stronger, also provide consistent, straight corners. Simpson Strong-Tie offers wide variety angles in various sizes thicknesses handle light-duty jobs projects structural connection needed. Some bent (skewed) match project. For outdoor projects moisture present, use ZMAX zinc-coated connectors, provide extra resistance corrosion (look ""Z"" end model number).Versatile connector various 90 connections home repair projects Stronger angled nailing screw fastening alone Help ensure joints consistently straight strong Dimensions: 3 in. x 3 in. x 1-1/2 in.Made 12-Gauge steel Galvanized extra corrosion resistance Install 10d common nails #9 x 1-1/2 in. Strong-Drive SD screws"
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not angles make joints stronger, also provide consistent, straight corners. Simpson Strong-Tie offers wide variety angles in various sizes thicknesses handle light-duty jobs projects structural connection needed. Some bent (skewed) match project. For outdoor projects moisture present, use ZMAX zinc-coated connectors, provide extra resistance corrosion (look ""Z"" end model number).Versatile connector various 90 connections home repair projects Stronger angled nailing screw fastening alone Help ensure joints consistently straight strong Dimensions: 3 in. x 3 in. x 1-1/2 in.Made 12-Gauge steel Galvanized extra corrosion resistance Install 10d common nails #9 x 1-1/2 in. Strong-Drive SD screws"
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 Tugboat Wood Concrete Coating,deck,3.00,"BEHR Premium Textured DECKOVER innovative solid color coating. It bring old, weathered wood concrete back life. The advanced 100% acrylic resin formula creates durable coating tired worn deck, rejuvenating whole new look. For best results, sure properly prepare surface using applicable BEHR products displayed above.California residents: see&nbsp;Proposition 65 information Revives wood composite decks, railings, porches boat docks, also great concrete pool decks, patios sidewalks100% acrylic solid color coating Resists cracking peeling conceals splinters cracks 1/4 in.Provides durable, mildew resistant finish Covers 75 sq. ft. in 2 coats per gallon Creates textured, slip-resistant finish For best results, prepare appropriate BEHR product wood concrete surface Actual paint colors may vary on-screen printer representations Colors available tinted in stores Online Price includes Paint Care fee in following states: CA, CO, CT, ME, MN, OR, RI, VT"
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Kit in Chrome,rain shower head,2.33,"Update bathroom Delta Vero Single-Handle Shower Faucet Trim Kit in Chrome. It sleek, modern minimalistic aesthetic. The Multi Choice universal valve keeps water temperature within +/-3 degrees Fahrenheit help prevent scalding.California residents: see&nbsp;Proposition 65 information Includes trim kit only, rough-in kit (R10000-UNBX) sold separately Includes handle Maintains balanced pressure hot cold water even valve turned elsewhere in system Due Water Sense regulations in state New York, please confirm shipping zip code restricted use items meet Water Sense qualifications"
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Kit in Chrome,shower faucet,2.67,"Update bathroom Delta Vero Single-Handle Shower Faucet Trim Kit in Chrome. It sleek, modern minimalistic aesthetic. The Multi Choice universal valve keeps water temperature within +/-3 degrees Fahrenheit help prevent scalding.California residents: see&nbsp;Proposition 65 information Includes trim kit only, rough-in kit (R10000-UNBX) sold separately Includes handle Maintains balanced pressure hot cold water even valve turned elsewhere in system Due Water Sense regulations in state New York, please confirm shipping zip code restricted use items meet Water Sense qualifications"

In [55]:
# Adds stemmed words

stemmer = SnowballStemmer('english')

def str_stemmer(s):
	if type(s) == str:
		return " ".join([stemmer.stem(word) for word in s.lower().split()])
	else:
		return


# Changed this function to handle NaN values
def str_common_word(str1, str2):
	if type(str1) == str and type(str2) == str:
		return sum(int(str2.find(word)>=0) for word in str1.split())
	else:
		return


# STEM THE ATTRIBUTES WE WANT TO TURN INTO COUNT FEATURES
df_all['search_term_stem'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title_stem'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description_stem'] = df_all['product_description'].map(lambda x:str_stemmer(x))

# COMBINE ALL STEMMED ATTRIBUTES INTO ONE COLUMN
df_all['product_info_stemmed'] = df_all['search_term_stem']+"\t"+df_all['product_title_stem']+"\t"+df_all['product_description_stem']

# COMBINE ALL UNSTEMMED ATTRIBUTES INTO ONE COLUMN
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']

In [56]:
df_all.head()

,id,product_uid,product_title,search_term,relevance,product_description,search_term_stem,product_title_stem,product_description_stem,product_info_stemmed,product_info
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not angles make joints stronger, also provide consistent, straight corners. Simpson Strong-Tie offers wide variety angles in various sizes thicknesses handle light-duty jobs projects structural connection needed. Some bent (skewed) match project. For outdoor projects moisture present, use ZMAX zinc-coated connectors, provide extra resistance corrosion (look ""Z"" end model number).Versatile connector various 90 connections home repair projects Stronger angled nailing screw fastening alone Help ensure joints consistently straight strong Dimensions: 3 in. x 3 in. x 1-1/2 in.Made 12-Gauge steel Galvanized extra corrosion resistance Install 10d common nails #9 x 1-1/2 in. Strong-Drive SD screws",angl bracket,simpson strong-ti 12-gaug angl,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl in various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair project stronger angl nail screw fasten alon help ensur joint consist straight strong dimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steel galvan extra corros resist instal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","angl bracket\tsimpson strong-ti 12-gaug angl\tnot angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl in various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair project stronger angl nail screw fasten alon help ensur joint consist straight strong dimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steel galvan extra corros resist instal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","angle bracket\tSimpson Strong-Tie 12-Gauge Angle\tNot angles make joints stronger, also provide consistent, straight corners. Simpson Strong-Tie offers wide variety angles in various sizes thicknesses handle light-duty jobs projects structural connection needed. Some bent (skewed) match project. For outdoor projects moisture present, use ZMAX zinc-coated connectors, provide extra resistance corrosion (look ""Z"" end model number).Versatile connector various 90 connections home repair projects Stronger angled nailing screw fastening alone Help ensure joints consistently straight strong Dimensions: 3 in. x 3 in. x 1-1/2 in.Made 12-Gauge steel Galvanized extra corrosion resistance Install 10d common nails #9 x 1-1/2 in. Strong-Drive SD screws"
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not angles make joints stronger, also provide consistent, straight corners. Simpson Strong-Tie offers wide variety angles in various sizes thicknesses handle light-duty jobs projects structural connection needed. Some bent (skewed) match project. For outdoor projects moisture present, use ZMAX zinc-coated connectors, provide extra resistance corrosion (look ""Z"" end model number).Versatile connector various 90 connections home repair projects Stronger angled nailing screw fastening alone Help ensure joints consistently straight strong Dimensions: 3 in. x 3 in. x 1-1/2 in.Made 12-Gauge steel Galvanized extra corrosion resistance Install 10d common nails #9 x 1-1/2 in. Strong-Drive SD screws",l bracket,simpson strong-ti 12-gaug angl,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl in various size thick handl light-duti job project 

In [57]:
def convert_to_lowercase(df_all):
    
    for column in df_all.columns:
        if column != 'id' and column != 'product_uid' and column != 'relevance':
            df_all[column] = df_all[column].str.lower()
    
    return df_all

In [58]:
df_all = convert_to_lowercase(df_all)

In [59]:
df_all.head()

,id,product_uid,product_title,search_term,relevance,product_description,search_term_stem,product_title_stem,product_description_stem,product_info_stemmed,product_info
0,2,100001,simpson strong-tie 12-gauge angle,angle bracket,3.00,"not angles make joints stronger, also provide consistent, straight corners. simpson strong-tie offers wide variety angles in various sizes thicknesses handle light-duty jobs projects structural connection needed. some bent (skewed) match project. for outdoor projects moisture present, use zmax zinc-coated connectors, provide extra resistance corrosion (look ""z"" end model number).versatile connector various 90 connections home repair projects stronger angled nailing screw fastening alone help ensure joints consistently straight strong dimensions: 3 in. x 3 in. x 1-1/2 in.made 12-gauge steel galvanized extra corrosion resistance install 10d common nails #9 x 1-1/2 in. strong-drive sd screws",angl bracket,simpson strong-ti 12-gaug angl,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl in various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair project stronger angl nail screw fasten alon help ensur joint consist straight strong dimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steel galvan extra corros resist instal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","angl bracket\tsimpson strong-ti 12-gaug angl\tnot angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl in various size thick handl light-duti job project structur connect needed. some bent (skewed) match project. for outdoor project moistur present, use zmax zinc-coat connectors, provid extra resist corros (look ""z"" end model number).versatil connector various 90 connect home repair project stronger angl nail screw fasten alon help ensur joint consist straight strong dimensions: 3 in. x 3 in. x 1-1/2 in.mad 12-gaug steel galvan extra corros resist instal 10d common nail #9 x 1-1/2 in. strong-driv sd screw","angle bracket\tsimpson strong-tie 12-gauge angle\tnot angles make joints stronger, also provide consistent, straight corners. simpson strong-tie offers wide variety angles in various sizes thicknesses handle light-duty jobs projects structural connection needed. some bent (skewed) match project. for outdoor projects moisture present, use zmax zinc-coated connectors, provide extra resistance corrosion (look ""z"" end model number).versatile connector various 90 connections home repair projects stronger angled nailing screw fastening alone help ensure joints consistently straight strong dimensions: 3 in. x 3 in. x 1-1/2 in.made 12-gauge steel galvanized extra corrosion resistance install 10d common nails #9 x 1-1/2 in. strong-drive sd screws"
1,3,100001,simpson strong-tie 12-gauge angle,l bracket,2.50,"not angles make joints stronger, also provide consistent, straight corners. simpson strong-tie offers wide variety angles in various sizes thicknesses handle light-duty jobs projects structural connection needed. some bent (skewed) match project. for outdoor projects moisture present, use zmax zinc-coated connectors, provide extra resistance corrosion (look ""z"" end model number).versatile connector various 90 connections home repair projects stronger angled nailing screw fastening alone help ensure joints consistently straight strong dimensions: 3 in. x 3 in. x 1-1/2 in.made 12-gauge steel galvanized extra corrosion resistance install 10d common nails #9 x 1-1/2 in. strong-drive sd screws",l bracket,simpson strong-ti 12-gaug angl,"not angl make joint stronger, also provid consistent, straight corners. simpson strong-ti offer wide varieti angl in various size thick handl light-duti job project 

In [60]:
# This adds attributes (name, value, all, values, search_term_and_all_attributes)

# ADDING ATTRIBUTES - COUNT OF PRODUCT ATTRIBUTES IN THE SEARCH TERM
attributes = df_attr.copy()
df_attr['all_values'] = df_attr[['product_uid', 'value']].groupby(['product_uid'])['value'].transform(lambda x : ','.join((word_tokenize(stemmer.stem(str(x))))))
df_all_w_attr = df_all.join(df_attr, how = 'left', lsuffix = '_left', rsuffix='_right')
df_all_w_attr['search_term_and_all_attributes'] = df_all_w_attr['search_term']+"\t"+df_all_w_attr['name'].transform(lambda x : ','.join(word_tokenize(str(x))))+"\t"+df_all_w_attr['all_values']

In [61]:
df_all_w_attr.columns

Index(['id', 'product_uid_left', 'product_title', 'search_term', 'relevance',
       'product_description', 'search_term_stem', 'product_title_stem',
       'product_description_stem', 'product_info_stemmed', 'product_info',
       'product_uid_right', 'name', 'value', 'all_values',
       'search_term_and_all_attributes'],
      dtype='object')

In [62]:
# CODE BLOCK 5: ATTRIBUTE - VALUE PAIRS PRE-PROCESSING
attributes = attributes.astype({"name":str})
grouped_attr = attributes.groupby(["product_uid", "name"], group_keys=False).agg(lambda x: x)
grouped_attr = grouped_attr.reset_index()
grouped_attr.columns = ["product_uid", "attr_name", "attr_value"]

In [63]:
# CODE BLOCK 7: EXTRACT MOST COMMON ATTRIBUTES

types_of_attributes = grouped_attr["attr_name"].value_counts()
types_of_attributes = types_of_attributes.to_frame()
types_of_attributes = types_of_attributes.reset_index()
bullets = types_of_attributes["attr_name"].str.contains("Bullet")
types_of_attributes = types_of_attributes[~bullets]

most_used_attributes = types_of_attributes.iloc[0:20]
most_used_attribute_list = []

for i in most_used_attributes["attr_name"]:
    most_used_attribute_list.append(i)
    # print(len(most_used_attribute_list))


grouped_attr = grouped_attr[grouped_attr["attr_name"].isin(most_used_attribute_list)]
attributes_as_cols = grouped_attr.pivot(index = "product_uid", columns="attr_name", values="attr_value")

In [64]:
attributes_as_cols.head()

attr_name,Assembled Depth (in.),Assembled Height (in.),Assembled Width (in.),Certifications and Listings,Color Family,Color/Finish,Color/Finish Family,Features,Hardware Included,Included,Interior/Exterior,MFG Brand Name,Material,Package Quantity,Product Depth (in.),Product Height (in.),Product Length (in.),Product Weight (lb.),Product Width (in.),Wattage (watts)
product_uid,,,,,,,,,,,,,,,,,,,,
100001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Simpson Strong-Tie,Galvanized Steel,NaN,1.5,3,NaN,0.26,3,NaN
100002.0,6.63 in,7.76 in,6.63 in,NaN,Browns / Tans,Tugboat,NaN,NaN,NaN,NaN,Exterior,BEHR Premium Textured DeckOver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003.0,NaN,NaN,NaN,"ANSI Certified,ASTM Compliant,CSA Certified",White,White,NaN,NaN,NaN,NaN,NaN,STERLING,Composite,NaN,NaN,NaN,NaN,129,NaN,NaN
100004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grape Solar,NaN,NaN,64.96,1.57,NaN,NaN,39,265
100005.0,NaN,NaN,NaN,"ADA Compliant,CSA Certified,IAPMO Certified",Chrome,Chrome,NaN,NaN,NaN,NaN,NaN,Delta,NaN,NaN,15.28,24,NaN,NaN,7.09,NaN


In [65]:
attributes_as_cols.columns

Index(['Assembled Depth (in.)', 'Assembled Height (in.)',
       'Assembled Width (in.)', 'Certifications and Listings', 'Color Family',
       'Color/Finish', 'Color/Finish Family', 'Features', 'Hardware Included',
       'Included', 'Interior/Exterior', 'MFG Brand Name', 'Material',
       'Package Quantity', 'Product Depth (in.)', 'Product Height (in.)',
       'Product Length (in.)', 'Product Weight (lb.)', 'Product Width (in.)',
       'Wattage (watts)'],
      dtype='object', name='attr_name')

In [66]:
# CODE BLOCK 8: PROCESS MEASUREMENT FEATURES

def cleanup_text_in_df(text, dataframe, column):
    dataframe[column] = dataframe[column].str.replace(text, '')
    return dataframe


columns_to_clean = ["Assembled Height (in.)", "Assembled Width (in.)", "Product Depth (in.)", "Product Height (in.)", "Product Length (in.)", "Product Width (in.)"]
attributes_as_cols = cleanup_text_in_df("in", attributes_as_cols, "Assembled Depth (in.)")
for column in columns_to_clean:
    attributes_as_cols = cleanup_text_in_df("in", attributes_as_cols, column)


def make_one_measurement(depth_col, width_col, length_col):
    def check_nan(col_name):
        nan = type(float('nan'))
        if type(col_name) != nan:
            return True
    measurements = []
    if check_nan(depth_col):
        depth = depth_col
        measurements.append(depth)
    if check_nan(width_col):
        width = width_col
        measurements.append(width)
    if check_nan(length_col):
        length = length_col
        measurements.append(length)
    measurement_str = (" x ").join(measurements)
    return measurement_str


attributes_as_cols["Assembled Measurement (DxWxH)"] = attributes_as_cols.apply(lambda x: make_one_measurement(x["Assembled Depth (in.)"], x["Assembled Height (in.)"], x["Assembled Width (in.)"]), axis = 1)
attributes_as_cols["Product Measurement (LxWxD)"] = attributes_as_cols.apply(lambda x: make_one_measurement(x["Product Length (in.)"], x["Product Width (in.)"], x["Assembled Depth (in.)"]), axis = 1)
attributes_as_cols["Product Measurement (DxWxH)"] = attributes_as_cols.apply(lambda x: make_one_measurement(x["Product Length (in.)"], x["Product Width (in.)"], x["Assembled Height (in.)"]), axis = 1)
attributes_as_cols.to_csv("attributes_as_cols.csv")

In [67]:
attributes_as_cols.columns

Index(['Assembled Depth (in.)', 'Assembled Height (in.)',
       'Assembled Width (in.)', 'Certifications and Listings', 'Color Family',
       'Color/Finish', 'Color/Finish Family', 'Features', 'Hardware Included',
       'Included', 'Interior/Exterior', 'MFG Brand Name', 'Material',
       'Package Quantity', 'Product Depth (in.)', 'Product Height (in.)',
       'Product Length (in.)', 'Product Weight (lb.)', 'Product Width (in.)',
       'Wattage (watts)', 'Assembled Measurement (DxWxH)',
       'Product Measurement (LxWxD)', 'Product Measurement (DxWxH)'],
      dtype='object', name='attr_name')

In [68]:
# CODE BLOCK 10: MAKE PRODUCT TITLE + MEASUREMENTS FEATURE

measurements_in_product = attributes_as_cols[["Assembled Measurement (DxWxH)", "Product Measurement (LxWxD)", "Product Measurement (DxWxH)"]]
measurements_in_product.reset_index(inplace = True)
measurements_in_product.columns = ["product_uid", "Assembled Measurement (DxWxH)", "Product Measurement (LxWxD)", "Product Measurement (DxWxH)"]
df_all_w_attr = df_all_w_attr.join(measurements_in_product, how = 'left')

# Unstemmed
df_all_w_attr["title_and_assembled_measurement"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Assembled Measurement (DxWxH)"]
df_all_w_attr["title_and_measurement_LWD"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Product Measurement (LxWxD)"]
df_all_w_attr["title_and_measurement_DWH"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Product Measurement (DxWxH)"]

# Stemmed
df_all_w_attr["title_and_assembled_measurement_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Assembled Measurement (DxWxH)"]
df_all_w_attr["title_and_measurement_LWD_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Product Measurement (LxWxD)"]
df_all_w_attr["title_and_measurement_DWH_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Product Measurement (DxWxH)"]

In [69]:
df_all_w_attr.columns

Index(['id', 'product_uid_left', 'product_title', 'search_term', 'relevance',
       'product_description', 'search_term_stem', 'product_title_stem',
       'product_description_stem', 'product_info_stemmed', 'product_info',
       'product_uid_right', 'name', 'value', 'all_values',
       'search_term_and_all_attributes', 'product_uid',
       'Assembled Measurement (DxWxH)', 'Product Measurement (LxWxD)',
       'Product Measurement (DxWxH)', 'title_and_assembled_measurement',
       'title_and_measurement_LWD', 'title_and_measurement_DWH',
       'title_and_assembled_measurement_stem',
       'title_and_measurement_LWD_stem', 'title_and_measurement_DWH_stem'],
      dtype='object')

In [70]:
# CODE BLOCK 11: MAKE PRODUCT TITLE + COLOUR FEATURE
colours = attributes_as_cols[["Color Family", "Color/Finish"]]
colours = colours.reset_index()
# Make Colour Stems
colours['Color Family Stem'] = colours['Color Family'].map(lambda x:str_stemmer(x))
colours['Color/Finish Stem'] = colours['Color/Finish'].map(lambda x:str_stemmer(x))

df_all_w_attr = df_all_w_attr.merge(colours, how = "left", on = "product_uid")
# Non-stemmed
df_all_w_attr["title_and_color"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Color/Finish"]
df_all_w_attr["title_and_color_fam"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Color Family"]
# Stemmed
df_all_w_attr["title_and_color_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Color/Finish Stem"]
df_all_w_attr["title_and_color_fam_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Color Family Stem"]


In [71]:
df_all_w_attr.columns

Index(['id', 'product_uid_left', 'product_title', 'search_term', 'relevance',
       'product_description', 'search_term_stem', 'product_title_stem',
       'product_description_stem', 'product_info_stemmed', 'product_info',
       'product_uid_right', 'name', 'value', 'all_values',
       'search_term_and_all_attributes', 'product_uid',
       'Assembled Measurement (DxWxH)', 'Product Measurement (LxWxD)',
       'Product Measurement (DxWxH)', 'title_and_assembled_measurement',
       'title_and_measurement_LWD', 'title_and_measurement_DWH',
       'title_and_assembled_measurement_stem',
       'title_and_measurement_LWD_stem', 'title_and_measurement_DWH_stem',
       'Color Family', 'Color/Finish', 'Color Family Stem',
       'Color/Finish Stem', 'title_and_color', 'title_and_color_fam',
       'title_and_color_stem', 'title_and_color_fam_stem'],
      dtype='object')

In [72]:
df_all_w_attr.iloc[1]

id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            3
product_uid_left                                                                                                                                                                                                        

In [73]:
# lppd = pd.read_csv("full preprocessed data.csv", index_col=0)

C:\Users\spiro\AppData\Local\Temp\ipykernel_11872\3715698908.py:1: DtypeWarning: Columns (6,9,10,11,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  lppd = pd.read_csv("full preprocessed data.csv", index_col=0)


In [79]:
print(attributes_as_cols.columns)
print(df_all_w_attr.columns)

Index(['Assembled Depth (in.)', 'Assembled Height (in.)',
       'Assembled Width (in.)', 'Certifications and Listings', 'Color Family',
       'Color/Finish', 'Color/Finish Family', 'Features', 'Hardware Included',
       'Included', 'Interior/Exterior', 'MFG Brand Name', 'Material',
       'Package Quantity', 'Product Depth (in.)', 'Product Height (in.)',
       'Product Length (in.)', 'Product Weight (lb.)', 'Product Width (in.)',
       'Wattage (watts)', 'Assembled Measurement (DxWxH)',
       'Product Measurement (LxWxD)', 'Product Measurement (DxWxH)'],
      dtype='object', name='attr_name')
Index(['id', 'product_uid_left', 'product_title', 'search_term', 'relevance',
       'product_description', 'search_term_stem', 'product_title_stem',
       'product_description_stem', 'product_info_stemmed', 'product_info',
       'product_uid_right', 'name', 'value', 'all_values',
       'search_term_and_all_attributes', 'product_uid',
       'Assembled Measurement (DxWxH)', 'Product Meas